#### 2023-04-28

##### 1교시 

In [ ]:
# sklearn LGBMClassifier에서 과적합을 방지하기 위해 높여야 하는 하이퍼파라미터는 다음 중 무엇인가?

In [ ]:
# 그리드 서치 대신 hyperopt로 배이지안 최적화를 이용한 하이퍼파라미터 튜닝을 하는 이유를 서술하시오

In [ ]:
# hyperopt에서 -1을 곱하는 이유는 무엇인가요?

In [ ]:
# 다음의 경우와 같은 실제 경우와 그에 대한 예측이 있을 때 정밀도와 재현율을 구하세요

In [2]:
# 다음 중 앙상블 모델이 아닌 것은?

In [ ]:
# 손실함수와 같은 의미의 용어를 쓰세요

In [ ]:
# 두 방식의 차이점을 설명하시오

In [ ]:
# 앙상블 기법의 조건

In [ ]:
# {       } 안을 채워넣으세요

In [72]:
from hyperopt import hp

In [96]:
search_space = { 'x' : hp.quniform('x',0,20,1),
                 'y' : hp.quniform('y',-50,50,5)  }

In [97]:
search_space

{'x': <hyperopt.pyll.base.Apply at 0x1d66e0ba830>,
 'y': <hyperopt.pyll.base.Apply at 0x1d66e0bbe50>}

In [ ]:
# objective_func : x*y - x + y 함수를 만드세요

In [98]:
from hyperopt import STATUS_OK
def objective_func(search_space): 
      x=search_space['x']
      y=search_space['y']
      retval= x*y -x+y
      return retval 

In [ ]:
# fmin() 함수 인자들을 채우세요

In [99]:
from hyperopt import fmin, tpe, Trials

In [100]:
trials = Trials()
best_01 = fmin(fn= objective_func, 
            space= search_space, 
            algo= tpe.suggest,
            max_evals=5, 
            trials= trials)
best_01

100%|████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1000.74trial/s, best loss: -459.0]


{'x': 9.0, 'y': -45.0}

In [89]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from pydataset import data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import time
import xgboost
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm import plot_importance
import matplotlib.pyplot as plt

##### 2교시 279p 

In [104]:
df = pd.read_csv('creditcard.csv')

In [105]:
df.drop('Time',axis =1, inplace=True)

In [106]:
X = X_features = df.iloc[:,:-1]
y = y_labels = df.iloc[:,-1]

In [109]:
(y.value_counts() / len(y)) * 100

0    99.827251
1     0.172749
Name: Class, dtype: float64

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [123]:
lr_clf = LogisticRegression(max_iter = 1000)
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
pred_proba = lr_clf.predict_proba(X_test)[:,1]

In [116]:
accuracy_score(y_test, pred)

0.999204147794436

In [117]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
          F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc)) 

In [119]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [121]:
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
[[85284    12]
 [   56    91]]
정확도: 0.9992, 정밀도: 0.8835, 재현율: 0.6190,          F1: 0.7280, AUC:0.9688


In [126]:
# 인자로 사이킷런의 Estimator객체와, 학습/테스트 데이터 세트를 입력 받아서 학습/예측/평가 수행.
def get_model_train_eval(model, X_train=None, X_test=None, y_train=None, y_test=None):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    pred_proba = model.predict_proba(X_test)[:, 1]
    get_clf_eval(y_test, pred, pred_proba)

In [130]:
# LogisticRegression 사용한 경우
get_model_train_eval(lr_clf, X_train, X_test, y_train, y_test)

오차 행렬
[[85284    12]
 [   56    91]]
정확도: 0.9992, 정밀도: 0.8835, 재현율: 0.6190,          F1: 0.7280, AUC:0.9688


In [131]:
# LightGBM 사용한 경우
lgbm_clf = LGBMClassifier(n_estimators=1000,
                         num_leaves=64,
                         n_jobs=-1,
                         boost_from_average=False)


get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)

오차 행렬
[[85287     9]
 [   29   118]]
정확도: 0.9996, 정밀도: 0.9291, 재현율: 0.8027,          F1: 0.8613, AUC:0.9800


In [139]:
# 286p : 스케일링 StandardScaler 사용 -> 어느 한 칼럼의 데이터들이 마음에 들지 않아 바꿔주는 과정

In [140]:
# Amount 칼럼에 대하여 스케일링 StandardScaler 사용
from sklearn.preprocessing import StandardScaler

# 방법 1. training으로 fit 한 후, transform()
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df['Amount'].values.reshape(-1,1))

In [145]:
# 방법 2. 데이터 스케일링 후에, 분리
X = X_features = df.iloc[:,:-1]
y = y_labels = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [141]:
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,-0.350151,0
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,-0.254117,0
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,-0.081839,0
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,-0.313249,0


In [146]:
# LightGBM 사용한 경우
lgbm_clf = LGBMClassifier(n_estimators=1000,
                         num_leaves=64,
                         n_jobs=-1,
                         boost_from_average=False)


get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)

오차 행렬
[[85288     8]
 [   29   118]]
정확도: 0.9996, 정밀도: 0.9365, 재현율: 0.8027,          F1: 0.8645, AUC:0.9829


##### 3교시 287p 

In [148]:
# amount 칼럼에 대해 로그변환 사용

df = pd.read_csv('creditcard.csv')
df.drop('Time',axis =1, inplace=True)

In [150]:
df['Amount'] = np.log1p(df['Amount'])

In [151]:
# LightGBM 사용한 경우
lgbm_clf = LGBMClassifier(n_estimators=1000,
                         num_leaves=64,
                         n_jobs=-1,
                         boost_from_average=False)


get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)

오차 행렬
[[85288     8]
 [   29   118]]
정확도: 0.9996, 정밀도: 0.9365, 재현율: 0.8027,          F1: 0.8645, AUC:0.9829


In [153]:
# LogisticRegressions 사용한 경우
lr_clf = LogisticRegression(max_iter = 1000)
get_model_train_eval(lr_clf, X_train, X_test, y_train, y_test)

오차 행렬
[[85284    12]
 [   56    91]]
정확도: 0.9992, 정밀도: 0.8835, 재현율: 0.6190,          F1: 0.7280, AUC:0.9690


In [159]:
np.log10(10)+np.log10(100)

3.0

In [160]:
np.log(np.exp(1)) # np.log1p 랑 같은의미

1.0

In [165]:
np.log1p(0)

0.0

In [167]:
# 288p ~ 
# IQR ( inter quantile range) 
1/4 지점 : -1.5IQR
3/4 지점 : +1.5IQR

SyntaxError: invalid decimal literal (3602684786.py, line 3)

In [169]:
x = np.arange(1,201)

In [171]:
np.percentile(x,25)
np.percentile(x,3/4*100)

150.25

In [ ]:
# 이상치 연습
1 ~ 200

하위 이상치 x값
상위 이상치 x값
을 찾아주세요

이상치 : 3/4 지점 - 1/4 지점

In [179]:
print('하위 :', np.percentile(x,25)-(1.5*(np.percentile(x,75)-(np.percentile(x,25)))))
print('상위 :', np.percentile(x,75)+(1.5*(np.percentile(x,75)-(np.percentile(x,25)))))

하위 : -98.5
상위 : 299.5


In [182]:
df = pd.read_csv('creditcard.csv')
df.drop('Time',axis =1, inplace=True)
df['Amount'].max()

25691.16

In [ ]:
# df['Amount'] 의 하위 이상치, 상위 이상치의 갯수는?

In [184]:
IQR = np.percentile(df['Amount'], 75) - np.percentile(df['Amount'], 25)
print("하위 : ", np.percentile(df['Amount'],25) - 1.5*IQR) 
print("상위 : ", np.percentile(df['Amount'],75) + 1.5*IQR) 

하위 :  -101.7475
상위 :  184.5125


In [185]:
low = np.percentile(df['Amount'],25) - 1.5*IQR
high = np.percentile(df['Amount'],75) + 1.5*IQR

In [187]:
sum(df['Amount'] < low )

0

In [188]:
sum(df['Amount'] > high )

31904

##### 5교시

In [193]:
# 이상치 아닌 관측치 관찰
df_normal = df[df['Amount'] < np.percentile(x,75) + 1.5*IQR]

In [195]:
df_normal['Amount'] = np.log1p(df_normal['Amount'])     # 1. 로그 스케일링

y = df_normal.iloc[:, -1]
X = df_normal.iloc[:, : -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)     # 2. 데이터셋 분리

# 3. 평가지표 구하기
# LogisticRegression    
lr_clf = LogisticRegression(max_iter = 1000)
get_model_train_eval(lr_clf, X_train, X_test, y_train, y_test)
print("========================================================")
# LightGBM
lgbm_clf = LGBMClassifier(n_estimators = 1000, num_leaves = 64, n_jobs = -1,
                         boost_from_average = False)
get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test) 

C:\Users\user\AppData\Local\Temp\ipykernel_14644\3554993567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normal['Amount'] = np.log1p(df_normal['Amount'])


오차 행렬
[[78697    15]
 [   50    94]]
정확도: 0.9992, 정밀도: 0.8624, 재현율: 0.6528,          F1: 0.7431, AUC:0.9832
오차 행렬
[[78706     6]
 [   21   123]]
정확도: 0.9997, 정밀도: 0.9535, 재현율: 0.8542,          F1: 0.9011, AUC:0.9870


In [198]:
X.sample(frac=0.1)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
127421,-0.358284,1.152559,1.294548,0.061350,0.072125,-0.953804,0.785839,-0.114657,-0.502470,-0.539223,...,0.178713,-0.250572,-0.634429,-0.002175,0.346551,-0.132733,0.069603,0.246925,0.100464,1.270934
263881,-0.571229,1.325994,1.253170,4.037335,0.573932,3.267649,-0.911067,1.496866,-1.522247,1.415402,...,0.024638,0.348578,1.143326,-0.013162,-0.766468,-0.882032,0.533280,0.401678,0.178552,0.009901
103433,0.974613,-0.350143,0.360393,1.484236,-0.801989,-0.499769,-0.090965,0.172559,0.584740,0.157606,...,-0.214332,-0.053625,-0.378858,-0.112321,0.457133,0.517524,-0.369436,-0.021903,0.018810,1.722075
177964,1.898070,0.508306,-0.699630,3.625149,0.492123,0.111834,0.154839,0.049418,-1.084564,1.627896,...,-0.329156,-0.251223,-0.799220,0.312881,-0.441921,-0.275426,-0.339787,-0.047181,-0.057665,1.171486
96701,-2.801212,-2.196221,0.602145,-1.046421,-0.948530,0.362579,-1.280901,1.205613,-0.357712,-0.621788,...,0.076624,0.371625,0.587742,-0.253313,-0.765052,0.551721,-0.111257,-0.156501,-0.598191,1.777095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83780,1.182574,-0.215672,1.229069,0.870968,-1.162277,-0.322833,-0.680997,0.119513,1.053453,-0.215972,...,-0.154172,-0.101612,-0.136055,0.042174,0.411874,0.227370,0.319185,0.011759,0.025446,1.163669
28618,-5.519577,-6.991976,2.697300,-1.394760,5.913509,-4.919245,-4.284688,0.601913,2.312010,-1.480092,...,1.522020,0.677240,0.224681,0.840202,0.059830,0.808195,-0.221072,-0.356809,-0.037947,0.325890
204619,1.972728,-0.059865,-2.285701,0.387192,0.564709,-1.127841,0.540178,-0.403614,0.420034,-0.439407,...,-0.004740,-0.206781,-0.607929,0.012443,-0.779594,0.014809,0.555388,-0.098759,-0.038074,1.696306
24425,1.153606,0.115139,0.578718,0.509888,-0.377366,-0.324067,-0.140144,0.091139,-0.166348,0.070173,...,-0.142390,-0.167880,-0.517491,0.181334,0.207062,0.071329,0.095324,-0.019569,0.007092,1.018179


In [203]:
df_normal[df_normal['Class'] == 0].sample(frac=0.99).index

Int64Index([158717, 222039, 139029, 218192, 145746,  62593, 231415, 168532,
             60871, 284082,
            ...
            119652, 166635, 221042, 127521, 181259,  49336, 161613, 273930,
             96917, 247190],
           dtype='int64', length=259807)

In [ ]:
df_normal.drop(df_normal[df_normal['Class']== 0].sample(frac=0.99).index) 

In [211]:
# X 의 1%만 뽑아서 X라고 합시다
df = pd.read_csv("creditcard.csv")
df.drop('Time', axis =1, inplace = True)


df_normal = df[df['Amount'] < np.percentile(x, 75 ) + 1.5*IQR]
df_normal = df_normal.drop(df_normal[df_normal['Class']== 0].sample(frac=0.99).index)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X = df_normal.iloc[:, : -1]
y = df_normal.iloc[:, -1] 


# LogisticRegression
lr_clf = LogisticRegression(max_iter = 1000)
get_model_train_eval(lr_clf, X_train, X_test, y_train, y_test)
print("========================================================")
# LightGBM
lgbm_clf = LGBMClassifier(n_estimators = 1000, num_leaves = 64, n_jobs = -1,
                         boost_from_average = False)
get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test) 

오차 행렬
[[784   2]
 [ 13 115]]
정확도: 0.9836, 정밀도: 0.9829, 재현율: 0.8984,          F1: 0.9388, AUC:0.9817
오차 행렬
[[785   1]
 [ 13 115]]
정확도: 0.9847, 정밀도: 0.9914, 재현율: 0.8984,          F1: 0.9426, AUC:0.9848


In [ ]:
# 292p 

In [220]:
from imblearn.over_sampling import SMOTE

In [221]:
smote = SMOTE(random_state=0)
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)

In [222]:
y_train.value_counts()

0    1838
1     293
Name: Class, dtype: int64

In [223]:
y_train_over.value_counts()

0    1838
1    1838
Name: Class, dtype: int64

In [224]:
# LogisticRegression
lr_clf = LogisticRegression(max_iter = 1000)
get_model_train_eval(lr_clf, X_train_over, X_test, y_train_over, y_test)
print("========================================================")
# LightGBM
lgbm_clf = LGBMClassifier(n_estimators = 1000, num_leaves = 64, n_jobs = -1,
                         boost_from_average = False)
get_model_train_eval(lgbm_clf, X_train_over, X_test, y_train_over, y_test) 

오차 행렬
[[771  15]
 [ 12 116]]
정확도: 0.9705, 정밀도: 0.8855, 재현율: 0.9062,          F1: 0.8958, AUC:0.9828
오차 행렬
[[784   2]
 [ 13 115]]
정확도: 0.9836, 정밀도: 0.9829, 재현율: 0.8984,          F1: 0.9388, AUC:0.9819


In [ ]:
# 295p 스태킹 앙상블 (잘 쓰이진 않음)

##### 6교시

In [228]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train , X_test , y_train , y_test = train_test_split(X_data , y_label , test_size=0.2 , random_state=0)

In [231]:
# 개별 ML 모델을 위한 Classifier 생성.
knn_clf  = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

In [232]:
# 개별 모델들을 학습. 
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train , y_train)
dt_clf.fit(X_train , y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [234]:
# 학습된 개별 모델들이 각자 반환하는 예측 데이터 셋을 생성하고 개별 모델의 정확도 측정. 
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)


In [235]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
pred

array([[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
        0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
        1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
        0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,

In [236]:
pred.shape

(4, 114)

In [237]:
pred[:, :3]

array([[0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1]])

In [238]:
pred.T

array([[0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 1, 0, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1,

In [239]:
train = pred = pred.T

In [242]:
lr_final = LogisticRegression(C=10)
lr_final.fit(pred, y_test)
final_pred = lr_final.predict(pred)
final_pred

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1])

In [243]:
final_pred == y_test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [244]:
accuracy_score(y_test, final_pred)

0.9649122807017544

In [ ]:
# 300p   CV 세트 기반의 스태킹 -> 잘 안쓰임

In [247]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [248]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test,  7)    
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
RandomForestClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
DecisionTreeClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


In [249]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print('원본 학습 피처 데이터 Shape:',X_train.shape, '원본 테스트 피처 Shape:',X_test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape: (455, 30) 원본 테스트 피처 Shape: (114, 30)
스태킹 학습 피처 데이터 Shape: (455, 4) 스태킹 테스트 피처 데이터 Shape: (114, 4)


In [250]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, stack_final)))

최종 메타 모델의 예측 정확도: 0.9737
